<a href="https://colab.research.google.com/github/trishabhanu/Code-Editor/blob/main/quickstarts/TwelveLabs_Quickstart_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/TwelveLabs_Quickstart_Embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

# Create embeddings

This guide shows how to utilize the TwelveLabs Python SDK to create embeddings for your videos.

# Prerequisites
Before you begin, ensure the following prerequisites are met:

- [Sign up](https://playground.twelvelabs.io/) for a free account and obtain your API key from the [API Key](https://playground.twelvelabs.io/dashboard/api-key) page. No credit card is required to use the Free plan. This plan allows you to index up to 600 minutes of videos, which is sufficient for a small project.
- The videos you wish to upload must meet the requirements in the [Prerequisites](https://docs.twelvelabs.io/docs/create-video-embeddings#prerequisites) section of the **Create video embeddings** page.



# Procedure

## Install the TwelveLabs Python SDK

In [1]:
%pip install -U -q twelvelabs

## Configure your API key


In [2]:
# For Google Colab, store your API key as a Secret named `TL_API_KEY`. If you don't know how to create a Colab Secret, see https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75.

from google.colab import userdata
TL_API_KEY = userdata.get('TL_API_KEY')

# For other Python environments, you can use environment variables
# TL_API_KEY = os.environ.get('TL_API_KEY')

## **Generate Embeddings**
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.
Learn more in the [docs](https://docs.twelvelabs.io/docs/create-video-embeddings)


In [3]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 5):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} embedding_option={segment.embedding_option} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

client = TwelveLabs(api_key=TL_API_KEY)
task = client.embed.task.create(
    model_name="Marengo-retrieval-2.7",
    video_url="https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_2mb.mp4" # # Example: https://sample-videos.com/video321/mp4/720/big_buck_bunny_720p_2mb.mp4
)
print(
    f"Created task: id={task.id} model_name={task.model_name} status={task.status}"
)

def on_task_update(task: EmbeddingsTask):
    print(f"  Status={task.status}")

status = task.wait_for_done(
    sleep_interval=2,
    callback=on_task_update
)
print(f"Embedding done: {status}")

task = task.retrieve(embedding_option=["visual-text", "audio"])
if task.video_embedding is not None and task.video_embedding.segments is not None:
    print_segments(task.video_embedding.segments)


Created task: id=6826ce66a1ad1e4e3b23530f model_name=Marengo-retrieval-2.7 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=ready
Embedding done: ready
  embedding_scope=clip embedding_option=visual-text start_offset_sec=0.0 end_offset_sec=6.0
  embeddings: [0.0025792755, 0.014416745, 0.013344925, -0.023667203, -0.034136135]
  embedding_scope=clip embedding_option=visual-text start_offset_sec=6.0 end_offset_sec=12.0
  embeddings: [0.014718804, 0.0048405766, 0.021370944, -0.029890832, -0.02597869]
  embedding_scope=clip embedding_option=audio start_offset_sec=0.0 end_offset_sec=6.0
  embeddings: [-0.05493164, -0.010864258, -0.031982422, -0.05053711, -0.020263672]
  embedding_scope=clip embedding_option=audio start